In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Testing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 19:58:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/26 19:58:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/26 19:58:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/12/26 19:58:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/12/26 19:58:18 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/12/26 19:58:18 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
/Users/aakashbolisetty/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureW

In [5]:
training = spark.read.csv('new_data.csv',header=True,inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols = ['age','Experience'],outputCol='IndependentFeatures')

In [11]:
output = featureAssembler.transform(training)
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|IndependentFeatures|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [12]:
final_data = output.select('IndependentFeatures','Salary')
final_data.show()

+-------------------+------+
|IndependentFeatures|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = final_data.randomSplit([0.75,0.25])
regression = LinearRegression(featuresCol='IndependentFeatures',labelCol='Salary')
regression  = regression.fit(train_data)

24/12/26 20:11:44 WARN Instrumentation: [43fa6fef] regParam is zero, which might cause numerical instability and overfitting.
24/12/26 20:11:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/26 20:11:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
regression.coefficients

DenseVector([-102.53, 1688.6818])

In [16]:
regression.intercept

16470.03994673731

In [17]:
##Predcition
predictions = regression.evaluate(test_data)
predictions.predictions.show()

+-------------------+------+-----------------+
|IndependentFeatures|Salary|       prediction|
+-------------------+------+-----------------+
|         [30.0,8.0]| 25000|26903.59520639148|
+-------------------+------+-----------------+

